In [1]:
from redis import Redis
from kafka import KafkaProducer
import json
import time

# Configuration for Redis and Kafka
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_CHANNEL = "your_redis_channel"
REDIS_STORAGE_KEY = "persistent_storage"

KAFKA_BROKER = "localhost:9092"
KAFKA_TOPIC = "CreditCardTransaction"

# Initialize Redis client
redis_client = Redis(host=REDIS_HOST, port=REDIS_PORT)

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
)

In [2]:
def stream_from_redis_to_kafka():
    """Stream data from Redis to Kafka."""
    pubsub = redis_client.pubsub()
    pubsub.subscribe(REDIS_CHANNEL)
    print(f"Subscribed to Redis channel: {REDIS_CHANNEL}")

    for message in pubsub.listen():
        if message["type"] == "message":
            data = message["data"].decode("utf-8")
            print(f"Received message from Redis: {data}")

            # Send the message to Kafka
            producer.send(KAFKA_TOPIC, {"message": data})
            print(f"Sent message to Kafka topic: {KAFKA_TOPIC}")

In [3]:
def list_redis_channels():
    """List all active channels in Redis."""
    channels = redis_client.pubsub_channels()
    print("Active Redis channels:")
    for channel in channels:
        print(channel.decode("utf-8"))

In [2]:
def write_to_redis_channels():
    """Write data to Redis channels and persist in storage."""
    try:
        while True:
            channel = input("Enter Redis channel name (or 'exit' to stop): ")
            if channel.lower() == "exit":
                break
            message = input(f"Enter message for channel '{channel}': ")

            # Persist the message in Redis storage (list)
            redis_client.rpush(
                REDIS_STORAGE_KEY, json.dumps({"channel": channel, "message": message})
            )
            print(f"Stored message in Redis key '{REDIS_STORAGE_KEY}': {message}")

            # Publish the message to the channel
            redis_client.publish(channel, message)
            print(f"Published message to Redis channel '{channel}': {message}")
    except KeyboardInterrupt:
        print("Stopping the Redis writing loop.")


def retrieve_persistent_data():
    """Retrieve and display all stored data from Redis."""
    try:
        data = redis_client.lrange(REDIS_STORAGE_KEY, 0, -1)
        print("Stored messages:")
        for item in data:
            print(json.loads(item))
    except Exception as e:
        print(f"An error occurred while retrieving data: {e}")


if __name__ == "__main__":
    try:
        print("Listing all Redis channels:")
        list_redis_channels()
        retrieve_persistent_data()
        write_to_redis_channels()
        stream_from_redis_to_kafka()
    except KeyboardInterrupt:
        print("Stopping the process.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
# exists = redis_client.exists("name")
# print(exists)  # Output: 1 (if exists), 0 (if not exists)

In [4]:
# redis_client.delete("tasks")

In [5]:
# redis_client.rpush('tasks','task5','task6')
# tasks = redis_client.lrange("tasks", 0, -1)
# print([task.decode() for task in tasks])  # Output: ['task1', 'task2', 'task3']

In [6]:
# redis_client.sadd("skills", "Python", "SQL", "Machine Learning")
# skills = redis_client.smembers("skills")
# print(
#     {skill.decode() for skill in skills}
# )  # Output: {'Python', 'SQL', 'Machine Learning'}

In [ ]:
redis_client.publish("notifications", "Hello, Redis!galal")

In [ ]:
pubsub = redis_client.pubsub()
pubsub.subscribe("notifications")

for message in pubsub.listen():
    print(message)  # Listen for published messages

In [5]:
x = "$61.72"
x[1:]

'61.72'

## sql

In [47]:
import pyodbc
import pandas as pd


def db_connection(
    server_name="DESKTOP-5DG0EQD\\SQLEXPRESS",
    database="SIA",
    username="Galal",
    password="123456789",
    Table="MCC_Categories",
):
    try:

        # Define your connection parameters
        server_name = server_name
        database = database
        username = username
        password = password

        # Create a connection to the database
        conn_str = f"DRIVER={{SQL Server}};SERVER={server_name};DATABASE={database};UID={username};PWD={password};"
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        query = f"SELECT * FROM {Table}"
        cursor.execute(query)
        rows = cursor.fetchall()

        columns = [column[0] for column in cursor.description]
        print(f"Database Schema: {columns}")

        df = pd.DataFrame.from_records(rows, columns=columns)

        cursor.close()
        conn.close()

        return df
    except Exception as e:
        print(f"Error connecting to the database: {e}")

In [55]:
print(db_connection().columns.values)

Database Schema: ['MCC', 'Category']
['MCC' 'Category']
